In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\pecci\Code\MACRO-dev\tutorials`


In [2]:
using Macro,CSV,DataFrames

Start MACRO model generation (units: MWh for energy, $ for costs)

In [3]:
T = 8760;
macro_settings = (Commodities = Dict(Electricity=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    Hydrogen=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    NaturalGas=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    CO2=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T)),
                PeriodLength = T);

In [4]:
macro_settings

(Commodities = Dict{DataType, Dict{Symbol, Int64}}(NaturalGas => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), Electricity => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), CO2 => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), Hydrogen => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1)), PeriodLength = 8760)

In [5]:

H2_MWh = 33.33 # MWh per tonne of H2
NG_MWh = 0.29307107 # MWh per MMBTU of NG

df = CSV.read("time_series_data.csv",DataFrame)

electricity_demand = df[1:T,:Electricity_Demand_MW]; # MWh
solar_capacity_factor = df[1:T,:Solar_Capacity_Factor]; # factor between 0 and 1
ng_fuel_price = df[1:T,:NG_Price]/NG_MWh; # $/MWh of natural gas
h2_demand = H2_MWh*df[1:T,:H2_Demand_tonne] # MWh of hydrogen

solar_inv_cost = 85300; # $/MW
solar_fom_cost = 18760.0; # $/MW

battery_inv_cost = 19584.0; #$/MW
battery_fom_cost = 4895; # $/MW
battery_vom_cost = 0.15; #$/MW
battery_inv_cost_storage = 22494.0; #$/MWh
battery_fom_cost_storage = 5622; #$/MWh
battery_vom_cost_storage = 0.15; #$/MWh
battery_eff_up = 0.92;
battery_eff_down = 0.92;

ngcc_inv_cost = 65400;# $/MW
ngcc_fom_cost = 10287.0;# $/MW
ngcc_vom_cost = 3.55; #$/MW
ngcc_capsize = 250.0;

ngcc_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of electricity
ngcc_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas

electrolyzer_capsize = 2*H2_MWh # MWh of H2
electrolyzer_efficiency = 1/(45/H2_MWh) # MWh of H2 / MWh of electricity
electrolyzer_inv_cost = 2033333/H2_MWh # $/MWh of H2
electrolyzer_fom_cost = 30500/H2_MWh # $/MWh of H2
electrolyzer_vom_cost = 0.0;


In [6]:
### For now, we use the same temporal resolution and subperiods for every commodity, but MACRO will allow to use different temporal resolution for different commodities

hours_per_timestep(c) = macro_settings.Commodities[c][:HoursPerTimeStep];
hours_per_subperiod(c) = macro_settings.Commodities[c][:HoursPerSubperiod]
time_interval(c)= 1:hours_per_timestep(c):macro_settings.PeriodLength

subperiods(c) = collect(Iterators.partition(time_interval(c), Int(hours_per_subperiod(c) / hours_per_timestep(c))),)

subperiods (generic function with 1 method)

Let's start from creating an electricity node:

In [7]:
e_node = Node{Electricity}(;
    id = Symbol("E_node"),
    demand =  electricity_demand,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    max_nsd = [0.0],
    price_nsd = [50000],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We assign a solar PV generator to this electricity node:

In [8]:
solar_pv = Resource{Electricity}(;
    id = :Solar_PV,
    node = e_node,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    capacity_factor = solar_capacity_factor,
    can_expand = true, 
    can_retire = false,
    existing_capacity = 0.0,
    investment_cost = solar_inv_cost,
    fixed_om_cost = solar_fom_cost,
    constraints = [Macro.CapacityConstraint()]
)

Resource{Electricity}(Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :Solar_PV, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1779, 0.429  …  0.3834, 0.2594, 0.078, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1.0, Float64[], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], 0.0, Inf, 0.0, true, false, 85300.0, 18760.0, 0.0, Dict{Symbol, Any}(), Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])

We can also assigne a battery storage to the node:

In [9]:
battery = SymmetricStorage{Electricity}(;
    id = :battery,
    node = e_node,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    can_expand = true,
    can_retire = false,
    existing_capacity = 0.0,
    investment_cost = battery_inv_cost,
    investment_cost_storage = battery_inv_cost_storage,
    fixed_om_cost = battery_fom_cost,
    fixed_om_cost_storage = battery_fom_cost_storage,
    variable_om_cost = battery_vom_cost,
    variable_om_cost_storage = battery_vom_cost_storage,
    efficiency_injection = battery_eff_down,
    efficiency_withdrawal = battery_eff_up,
    constraints = [Macro.CapacityConstraint(),Macro.StorageCapacityConstraint()]
)

SymmetricStorage{Electricity}(Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :battery, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 0.0, Inf, 0.0, Inf, 0.0, 0.0, true, false, 19584.0, 22494.0, 0.0, 4895.0, 5622.0, 0.0, 0.15, 0.15, 0.0, 0.92, 0.92, 0.0, 1.0, 10.0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing), Macro.StorageCapacityConstraint(missing, missing, missing)])

We also consider a hydrogen demand node:

In [10]:
h2_node = Node{Hydrogen}(;
    id = Symbol("H2_node"),
    demand =  h2_demand,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We model an exogenous inflow of natural gas into the system using by introducing a natural gas source node (with zero demand):

In [11]:
ng_node = Node{NaturalGas}(;
    id = Symbol("NG_node"),
    time_interval = time_interval(NaturalGas),
    subperiods = subperiods(NaturalGas),
    demand = zeros(length(time_interval(NaturalGas))),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

In [12]:
ng_source = Resource{NaturalGas}(;
id = :NG_Source,
node = ng_node,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
can_expand = false, 
can_retire = false,
price = ng_fuel_price,
existing_capacity = Inf,
)

Resource{NaturalGas}(Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :NG_Source, Float64[], 1.0, [18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264  …  14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], 0.0, Inf, Inf, false, false, 0.0, 0.0, 0.0, Dict{Symbol, Any}(), Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[])

In [13]:
co2_node = Node{CO2}(;
id = Symbol("CO2_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
demand = zeros(length(time_interval(CO2))),
max_nsd = [0.0],
price_nsd = [0.0],
constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We also define a CO2 sink resource (here we could add a CO2 cap):

In [14]:
co2_sink = Sink{CO2}(;
node = co2_node,
id = Symbol("CO2_sink"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
)

Sink{CO2}(Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :CO2_sink, Float64[], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[])

We are now ready to define a natural gas plant! Note that we require two stoichiometry balances:

Heat_Rate $\times$ Electricity = NaturalGas

Fuel_CO2 $\times$ NaturalGas = CO2

In [15]:
ngcc = Transformation{NaturalGasPower}(;
id = :NGCC,
time_interval = time_interval(Electricity),
number_of_stoichiometry_balances = 2,
constraints = [Macro.StoichiometryBalanceConstraint()]
)

Transformation{NaturalGasPower}(:NGCC, 1:1:8760, 2, TEdge[], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)])

This transformation has 3 edges: each edge carries a specific commodity flow

In [16]:
push!(ngcc.TEdges,TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = ngcc,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
capacity_size = ngcc_capsize,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
st_coeff = [ngcc_heatrate,0.0],
existing_capacity = 0.0,
investment_cost = ngcc_inv_cost,
fixed_om_cost = ngcc_fom_cost,
variable_om_cost = ngcc_vom_cost,
constraints = [CapacityConstraint()]
))

1-element Vector{TEdge}:
 TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPower}(:NGCC, 1:1:8760, 2, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 250.0, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0], 0.0, Inf, 0.0, 65400.0, 10287.0, 3.55, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])

In [17]:
push!(ngcc.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = ngcc,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,ngcc_fuel_CO2]
))

2-element Vector{TEdge}:
 TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPower}(:NGCC, 1:1:8760, 2, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 250.0, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0], 0.0, Inf, 0.0, 65400.0, 10287.0, 3.55, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])
 TEdge{N

In [18]:
push!(ngcc.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = ngcc,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0]
))

3-element Vector{TEdge}:
 TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPower}(:NGCC, 1:1:8760, 2, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 250.0, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0], 0.0, Inf, 0.0, 65400.0, 10287.0, 3.55, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])
 TEdge{N

We can also create an electrolyzer, which is defined by a single stoichiometry balance: electrolyzer_efficiency $\times$ Electricity = H2

In [19]:
electrolyzer = Transformation{Electrolyzer}(;
                id = :Electrolyzer,
                time_interval = time_interval(Electricity),
                number_of_stoichiometry_balances = 1,
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

Transformation{Electrolyzer}(:Electrolyzer, 1:1:8760, 1, TEdge[], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)])

In [20]:
push!(electrolyzer.TEdges,TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = electrolyzer,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electrolyzer_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = [1.0],
    existing_capacity = 0.0,
    investment_cost = electrolyzer_inv_cost,
    fixed_om_cost = electrolyzer_fom_cost,
    variable_om_cost = electrolyzer_vom_cost,
    constraints = [CapacityConstraint()]
))

1-element Vector{TEdge}:
 TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electrolyzer}(:Electrolyzer, 1:1:8760, 1, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 66.66, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0], 0.0, Inf, 0.0, 61006.090609060906, 915.0915091509152, 0.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConst

In [21]:
push!(electrolyzer.TEdges,TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electrolyzer,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = [electrolyzer_efficiency]
))

2-element Vector{TEdge}:
 TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electrolyzer}(:Electrolyzer, 1:1:8760, 1, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 66.66, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0], 0.0, Inf, 0.0, 61006.090609060906, 915.0915091509152, 0.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConst

Now that we have implemented different technologies, we can create the JuMP model.

In [22]:
components = [solar_pv; battery; ngcc; electrolyzer; ng_source;co2_sink];
nodes = [e_node;h2_node;ng_node;co2_node];
system = [nodes;components];

In [23]:
model = Macro.JuMP.Model();
Macro.@variable(model,vREF==1) ## Variable used to initialize empty expressions
Macro.@expression(model, eFixedCost, 0 * model[:vREF]);
Macro.@expression(model, eVariableCost, 0 * model[:vREF]);

In [24]:
add_planning_variables!.(components,model);

In [25]:
add_operation_variables!.(system, model);

In [26]:
add_all_model_constraints!.(system, model);

In [27]:
Macro.@objective(model, Min, model[:eFixedCost] + model[:eVariableCost]);


In [28]:
println(ngcc.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Electricity_E[5] - vFLOW_NaturalGas_NG[5] == 0


In [29]:
println(ngcc.constraints[1].constraint_ref[2,5])

-0.181048235160161 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] == 0


In [30]:
println(electrolyzer.constraints[1].constraint_ref[1,5])

vFLOW_Hydrogen_H2[5] - 0.7406666666666667 vFLOW_Electricity_E[5] == 0


In [31]:
using Gurobi

In [32]:
Macro.set_optimizer(model,Gurobi.Optimizer)

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2024-03-14


In [33]:
Macro.optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 148927 rows, 131416 columns and 328368 nonzeros
Model fingerprint: 0x7ad7410a
Coefficient statistics:
  Matrix range     [1e-04, 3e+02]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 2e+04]
Presolve removed 100896 rows and 92141 columns
Presolve time: 0.08s
Presolved: 48031 rows, 39275 columns, 126573 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 4
 AA' NZ     : 9.606e+04
 Factor NZ  : 6.263e+05 (roughly 40 MB of memory)
 Factor Ops : 8.892e+06 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual       

The installed solar capacity in MW is:

In [34]:
Macro.value(Macro.capacity(solar_pv))

0.0

The installed battery capacity in MW is:

In [35]:
Macro.value(Macro.capacity(solar_pv))

0.0

The installed NGCC capacity in MW is:

In [36]:
Macro.value(Macro.capacity(ngcc.TEdges[1]))

17831.1

The resulting CO2 emissions in tons are:

In [37]:
base_emissions  = Macro.value(sum(Macro.withdrawal(co2_sink)[t] for t in time_interval(CO2)))

3.491901634032986e7

However, if we add a CO2 cap:

In [38]:
Macro.@constraint(model,CO2Cap,sum(Macro.withdrawal(co2_sink)[t] for t in time_interval(CO2)) <= 0.5*base_emissions);

In [39]:
Macro.optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 148928 rows, 131416 columns and 337128 nonzeros
Coefficient statistics:
  Matrix range     [1e-04, 3e+02]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 2e+07]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.6842626e+09   1.745951e+07   0.000000e+00      0s
    2406    5.2759999e+09   2.428142e+08   0.000000e+00      5s
    4536    5.8096143e+09   2.958089e+08   0.000000e+00     10s
    6480    5.9199213e+09   1.076953e+08   0.000000e+00     15s
    9808    6.2467814e+09   7.237549e+07   0.000000e+00     20s
   11692    6.3392725e+09   8.326327e+07   0.000000e+00     25s
   13789    6.3768662e+09   1.252898e+06   0.000000e+00     31s
   15015    6.3844874e+09   0.00000

Now the solar capacity is:

In [40]:
Macro.value(Macro.capacity(solar_pv))

31896.409865489022

The installed battery capacity is:

In [41]:
Macro.value(Macro.capacity(battery))

5804.307789298313

The installed NG capacity is:

In [42]:
Macro.value(Macro.capacity(ngcc.TEdges[1]))

10568.542210701688

Let's validate the battery modeling:

In [44]:
diff_storage_level = zeros(T);
for t in 1:T
    if t==1
        diff_storage_level[t] = Macro.value(Macro.storage_level(battery)[1] - (1 - Macro.storage_loss_percentage(battery)) * Macro.storage_level(battery)[T])
    else
        diff_storage_level[t] = Macro.value(Macro.storage_level(battery)[t] - (1 - Macro.storage_loss_percentage(battery)) * Macro.storage_level(battery)[t-1])
    end
end

power_in = Macro.efficiency_withdrawal(battery)*Macro.value.(Macro.withdrawal(battery).data);
power_out = (1/Macro.efficiency_injection(battery))*Macro.value.(Macro.injection(battery).data);

In [45]:
maximum(abs.(diff_storage_level - (power_in - power_out)))

1.084998757505673e-10